# Loading Data

> Loading the data and the relevant groups 

- order: 1

In [ ]:
#| default_exp load

In [ ]:
#| hide
from __future__ import annotations

In [ ]:
#|export
from nbdev.showdoc import *
import nbdev
nbdev.nbdev_export()

In [ ]:
#| export
__version__ = "0.3.1"

In [ ]:
#| export
class Dabest(object):

    """
    Class for estimation statistics and plots.
    """

    def __init__(self, data, idx, x, y, paired, id_col, ci, 
                resamples, random_seed, proportional, delta2, 
                experiment, experiment_label, x1_level, mini_meta):

        """
        Parses and stores pandas DataFrames in preparation for estimation
        statistics. You should not be calling this class directly; instead,
        use `dabest.load()` to parse your DataFrame prior to analysis.
        """

        # Import standard data science libraries.
        import numpy as np
        import pandas as pd
        import seaborn as sns

        self.__delta2       = delta2
        self.__experiment   = experiment
        self.__ci           = ci
        self.__data         = data
        self.__id_col       = id_col
        self.__is_paired    = paired
        self.__resamples    = resamples
        self.__random_seed  = random_seed
        self.__proportional = proportional
        self.__mini_meta    = mini_meta 

        # Make a copy of the data, so we don't make alterations to it.
        data_in = data.copy()
        # data_in.reset_index(inplace=True)
        # data_in_index_name = data_in.index.name


        # Check if it is a valid mini_meta case
        if mini_meta is True:
            if proportional is True:
                err0 = '`proportional` and `mini_meta` cannot be True at the same time.'
                raise ValueError(err0)
            elif delta2 is True:
                err0 = '`delta` and `mini_meta` cannot be True at the same time.'
                raise ValueError(err0)
            
            if all([isinstance(i, str) for i in idx]):
                if len(pd.unique([t for t in idx]).tolist())!=2:
                    err0 = '`mini_meta` is True, but `idx` ({})'.format(idx) 
                    err1 = 'does not contain exactly 2 columns.'
                    raise ValueError(err0 + err1)
            elif all([isinstance(i, (tuple, list)) for i in idx]):
                all_idx_lengths = [len(t) for t in idx]
                if (np.array(all_idx_lengths) != 2).any():
                    err1 = "`mini_meta` is True, but some idx "
                    err2 = "in {} does not consist only of two groups.".format(idx)
                    raise ValueError(err1 + err2)
            


        # Check if this is a 2x2 ANOVA case and x & y are valid columns
        # Create experiment_label and x1_level
        if delta2 is True:
            if proportional is True:
                err0 = '`proportional` and `delta` cannot be True at the same time.'
                raise ValueError(err0)
            # idx should not be specified
            if idx:
                err0 = '`idx` should not be specified when `delta2` is True.'.format(len(x))
                raise ValueError(err0)

            # Check if x is valid
            if len(x) != 2:
                err0 = '`delta2` is True but the number of variables indicated by `x` is {}.'.format(len(x))
                raise ValueError(err0)
            else:
                for i in x:
                    if i not in data_in.columns:
                        err = '{0} is not a column in `data`. Please check.'.format(i)
                        raise IndexError(err)

            # Check if y is valid
            if not y:
                err0 = '`delta2` is True but `y` is not indicated.'
                raise ValueError(err0)
            elif y not in data_in.columns:
                err = '{0} is not a column in `data`. Please check.'.format(y)
                raise IndexError(err)

            # Check if experiment is valid
            if experiment not in data_in.columns:
                err = '{0} is not a column in `data`. Please check.'.format(experiment)
                raise IndexError(err)

            # Check if experiment_label is valid and create experiment when needed
            if experiment_label:
                if len(experiment_label) != 2:
                    err0 = '`experiment_label` does not have a length of 2.'
                    raise ValueError(err0)
                else: 
                    for i in experiment_label:
                        if i not in data_in[experiment].unique():
                            err = '{0} is not an element in the column `{1}` of `data`. Please check.'.format(i, experiment)
                            raise IndexError(err)
            else:
                experiment_label = data_in[experiment].unique()

            # Check if x1_level is valid
            if x1_level:
                if len(x1_level) != 2:
                    err0 = '`x1_level` does not have a length of 2.'
                    raise ValueError(err0)
                else: 
                    for i in x1_level:
                        if i not in data_in[x[0]].unique():
                            err = '{0} is not an element in the column `{1}` of `data`. Please check.'.format(i, experiment)
                            raise IndexError(err)

            else:
                x1_level = data_in[x[0]].unique()    
        self.__experiment_label = experiment_label
        self.__x1_level         = x1_level


        # Check if idx is specified
        if delta2 is False and not idx:
            err = '`idx` is not a column in `data`. Please check.'
            raise IndexError(err)


        # create new x & idx and record the second variable if this is a valid 2x2 ANOVA case
        if delta2 is True:
            # add a new column which is a combination of experiment and the first variable
            new_col_name = experiment+x[0]
            while new_col_name in data_in.columns:
                new_col_name += "_"
            data_in[new_col_name] = data_in[x[0]].astype(str) + " " + data_in[experiment].astype(str)

            #create idx            
            idx = []
            for i in list(map(lambda x: str(x), experiment_label)):
                temp = []
                for j in list(map(lambda x: str(x), x1_level)):
                    temp.append(j + " " + i)
                idx.append(temp)         
            self.__idx = idx
            self.__x1  = x[0]
            self.__x2  = x[1]
            # record the second variable and create idx
            x = new_col_name
        else:
            self.__idx = idx
            self.__x1  = None
            self.__x2  = None

        # Determine the kind of estimation plot we need to produce.
        if all([isinstance(i, str) for i in idx]):
            # flatten out idx.
            all_plot_groups = pd.unique([t for t in idx]).tolist()
            if len(idx) > len(all_plot_groups):
                err0 = '`idx` contains duplicated groups. Please remove any duplicates and try again.'
                raise ValueError(err0)
                
            # We need to re-wrap this idx inside another tuple so as to
            # easily loop thru each pairwise group later on.
            self.__idx = (idx,)

        elif all([isinstance(i, (tuple, list)) for i in idx]):
            all_plot_groups = pd.unique([tt for t in idx for tt in t]).tolist()
            
            actual_groups_given = sum([len(i) for i in idx])
            
            if actual_groups_given > len(all_plot_groups):
                err0 = 'Groups are repeated across tuples,'
                err1 = ' or a tuple has repeated groups in it.'
                err2 = ' Please remove any duplicates and try again.'
                raise ValueError(err0 + err1 + err2)

        else: # mix of string and tuple?
            err = 'There seems to be a problem with the idx you'
            'entered--{}.'.format(idx)
            raise ValueError(err)

        # Having parsed the idx, check if it is a kosher paired plot,
        # if so stated.
        #if paired is True:
        #    all_idx_lengths = [len(t) for t in self.__idx]
        #    if (np.array(all_idx_lengths) != 2).any():
        #        err1 = "`is_paired` is True, but some idx "
        #        err2 = "in {} does not consist only of two groups.".format(idx)
        #        raise ValueError(err1 + err2)

        # Check if there is a typo on paired
        if paired:
            if paired not in ("baseline", "sequential"):
                err = '{} assigned for `paired` is not valid.'.format(paired)
                raise ValueError(err)


        # Determine the type of data: wide or long.
        if x is None and y is not None:
            err = 'You have only specified `y`. Please also specify `x`.'
            raise ValueError(err)

        elif y is None and x is not None:
            err = 'You have only specified `x`. Please also specify `y`.'
            raise ValueError(err)

        # Identify the type of data that was passed in.
        elif x is not None and y is not None:
            # Assume we have a long dataset.
            # check both x and y are column names in data.
            if x not in data_in.columns:
                err = '{0} is not a column in `data`. Please check.'.format(x)
                raise IndexError(err)
            if y not in data_in.columns:
                err = '{0} is not a column in `data`. Please check.'.format(y)
                raise IndexError(err)

            # check y is numeric.
            if not np.issubdtype(data_in[y].dtype, np.number):
                err = '{0} is a column in `data`, but it is not numeric.'.format(y)
                raise ValueError(err)

            # check all the idx can be found in data_in[x]
            for g in all_plot_groups:
                if g not in data_in[x].unique():
                    err0 = '"{0}" is not a group in the column `{1}`.'.format(g, x)
                    err1 = " Please check `idx` and try again."
                    raise IndexError(err0 + err1)

            # Select only rows where the value in the `x` column 
            # is found in `idx`.
            plot_data = data_in[data_in.loc[:, x].isin(all_plot_groups)].copy()
            
            # plot_data.drop("index", inplace=True, axis=1)

            # Assign attributes
            self.__x = x
            self.__y = y
            self.__xvar = x
            self.__yvar = y

        elif x is None and y is None:
            # Assume we have a wide dataset.
            # Assign attributes appropriately.
            self.__x = None
            self.__y = None
            self.__xvar = "group"
            self.__yvar = "value"

            # First, check we have all columns in the dataset.
            for g in all_plot_groups:
                if g not in data_in.columns:
                    err0 = '"{0}" is not a column in `data`.'.format(g)
                    err1 = " Please check `idx` and try again."
                    raise IndexError(err0 + err1)
                    
            set_all_columns     = set(data_in.columns.tolist())
            set_all_plot_groups = set(all_plot_groups)
            id_vars = set_all_columns.difference(set_all_plot_groups)

            plot_data = pd.melt(data_in,
                                id_vars=id_vars,
                                value_vars=all_plot_groups,
                                value_name=self.__yvar,
                                var_name=self.__xvar)
                                
        # Added in v0.2.7.
        # remove any NA rows.
        plot_data.dropna(axis=0, how='any', subset=[self.__yvar], inplace=True)

        
        # Lines 131 to 140 added in v0.2.3.
        # Fixes a bug that jammed up when the xvar column was already 
        # a pandas Categorical. Now we check for this and act appropriately.
        if isinstance(plot_data[self.__xvar].dtype, 
                      pd.CategoricalDtype) is True:
            plot_data[self.__xvar].cat.remove_unused_categories(inplace=True)
            plot_data[self.__xvar].cat.reorder_categories(all_plot_groups, 
                                                          ordered=True, 
                                                          inplace=True)
        else:
            plot_data.loc[:, self.__xvar] = pd.Categorical(plot_data[self.__xvar],
                                               categories=all_plot_groups,
                                               ordered=True)
        
        # # The line below was added in v0.2.4, removed in v0.2.5.
        # plot_data.dropna(inplace=True)
        
        self.__plot_data = plot_data
        
        self.__all_plot_groups = all_plot_groups


        # Sanity check that all idxs are paired, if so desired.
        #if paired is True:
        #    if id_col is None:
        #        err = "`id_col` must be specified if `is_paired` is set to True."
        #        raise IndexError(err)
        #    elif id_col not in plot_data.columns:
        #        err = "{} is not a column in `data`. ".format(id_col)
        #        raise IndexError(err)

        # Check if `id_col` is valid
        if paired:
            if id_col is None:
                err = "`id_col` must be specified if `paired` is assigned with a not NoneType value."
                raise IndexError(err)
            elif id_col not in plot_data.columns:
                err = "{} is not a column in `data`. ".format(id_col)
                raise IndexError(err)

        EffectSizeDataFrame_kwargs = dict(ci=ci, is_paired=paired,
                                           random_seed=random_seed,
                                           resamples=resamples,
                                           proportional=proportional, 
                                           delta2=delta2, 
                                           experiment_label=self.__experiment_label,
                                           x1_level=self.__x1_level,
                                           x2=self.__x2,
                                           mini_meta = mini_meta)

        self.__mean_diff    = EffectSizeDataFrame(self, "mean_diff",
                                                **EffectSizeDataFrame_kwargs)

        self.__median_diff  = EffectSizeDataFrame(self, "median_diff",
                                               **EffectSizeDataFrame_kwargs)

        self.__cohens_d     = EffectSizeDataFrame(self, "cohens_d",
                                                **EffectSizeDataFrame_kwargs)

        self.__cohens_h     = EffectSizeDataFrame(self, "cohens_h",
                                                **EffectSizeDataFrame_kwargs)                                       

        self.__hedges_g     = EffectSizeDataFrame(self, "hedges_g",
                                                **EffectSizeDataFrame_kwargs)

        if not paired:
            self.__cliffs_delta = EffectSizeDataFrame(self, "cliffs_delta",
                                                    **EffectSizeDataFrame_kwargs)
        else:
            self.__cliffs_delta = "The data is paired; Cliff's delta is therefore undefined."


    def __repr__(self):
        # from .__init__ import __version__
        import datetime as dt
        import numpy as np

        # from .misc_tools import print_greeting

        # Removed due to the deprecation of is_paired
        #if self.__is_paired:
        #    es = "Paired e"
        #else:
        #    es = "E"

        greeting_header = print_greeting()

        RM_STATUS = {'baseline'  : 'for repeated measures against baseline \n', 
                     'sequential': 'for the sequential design of repeated-measures experiment \n',
                     'None'      : ''
                    }

        PAIRED_STATUS = {'baseline'   : 'Paired e', 
                         'sequential' : 'Paired e',
                         'None'       : 'E'
        }

        first_line = {"rm_status"    : RM_STATUS[str(self.__is_paired)],
                      "paired_status": PAIRED_STATUS[str(self.__is_paired)]}

        s1 = "{paired_status}ffect size(s) {rm_status}".format(**first_line)
        s2 = "with {}% confidence intervals will be computed for:".format(self.__ci)
        desc_line = s1 + s2

        out = [greeting_header + "\n\n" + desc_line]

        comparisons = []

        if self.__is_paired == 'sequential':
            for j, current_tuple in enumerate(self.__idx):
                for ix, test_name in enumerate(current_tuple[1:]):
                    control_name = current_tuple[ix]
                    comparisons.append("{} minus {}".format(test_name, control_name))
        else:
            for j, current_tuple in enumerate(self.__idx):
                control_name = current_tuple[0]

                for ix, test_name in enumerate(current_tuple[1:]):
                    comparisons.append("{} minus {}".format(test_name, control_name))

        if self.__delta2 is True:
            comparisons.append("{} minus {} (only for mean difference)".format(self.__experiment_label[1], self.__experiment_label[0]))
        
        if self.__mini_meta is True:
            comparisons.append("weighted delta (only for mean difference)")

        for j, g in enumerate(comparisons):
            out.append("{}. {}".format(j+1, g))

        resamples_line1 = "\n{} resamples ".format(self.__resamples)
        resamples_line2 = "will be used to generate the effect size bootstraps."
        out.append(resamples_line1 + resamples_line2)

        return "\n".join(out)


    # def __variable_name(self):
    #     return [k for k,v in locals().items() if v is self]
    #
    # @property
    # def variable_name(self):
    #     return self.__variable_name()
    
    @property
    def mean_diff(self):
        """
        Returns an :py:class:`EffectSizeDataFrame` for the mean difference, its confidence interval, and relevant statistics, for all comparisons as indicated via the `idx` and `paired` argument in `dabest.load()`.
        
        Example
        -------
        >>> from scipy.stats import norm
        >>> import pandas as pd
        >>> import dabest
        >>> control = norm.rvs(loc=0, size=30, random_state=12345)
        >>> test    = norm.rvs(loc=0.5, size=30, random_state=12345)
        >>> my_df   = pd.DataFrame({"control": control,
                                    "test": test})
        >>> my_dabest_object = dabest.load(my_df, idx=("control", "test"))
        >>> my_dabest_object.mean_diff
        
        Notes
        -----
        This is simply the mean of the control group subtracted from
        the mean of the test group.
        
        .. math::
            \\text{Mean difference} = \\overline{x}_{Test} - \\overline{x}_{Control}
            
        where :math:`\\overline{x}` is the mean for the group :math:`x`.
        """
        return self.__mean_diff
        
        
    @property    
    def median_diff(self):
        """
        Returns an :py:class:`EffectSizeDataFrame` for the median difference, its confidence interval, and relevant statistics, for all comparisons  as indicated via the `idx` and `paired` argument in `dabest.load()`.
        
        Example
        -------
        >>> from scipy.stats import norm
        >>> import pandas as pd
        >>> import dabest
        >>> control = norm.rvs(loc=0, size=30, random_state=12345)
        >>> test    = norm.rvs(loc=0.5, size=30, random_state=12345)
        >>> my_df   = pd.DataFrame({"control": control,
                                    "test": test})
        >>> my_dabest_object = dabest.load(my_df, idx=("control", "test"))
        >>> my_dabest_object.median_diff
        
        Notes
        -----
        This is the median difference between the control group and the test group.
        
        If the comparison(s) are unpaired, median_diff is computed with the following equation:

        .. math::
            \\text{Median difference} = \\widetilde{x}_{Test} - \\widetilde{x}_{Control}
            
        where :math:`\\widetilde{x}` is the median for the group :math:`x`.

        If the comparison(s) are paired, median_diff is computed with the following equation:

        .. math::
            \\text{Median difference} = \\widetilde{x}_{Test - Control}

        """
        return self.__median_diff
        
        
    @property
    def cohens_d(self):
        """
        Returns an :py:class:`EffectSizeDataFrame` for the standardized mean difference Cohen's `d`, its confidence interval, and relevant statistics, for all comparisons as indicated via the `idx` and `paired` argument in `dabest.load()`.
        
        Example
        -------
        >>> from scipy.stats import norm
        >>> import pandas as pd
        >>> import dabest
        >>> control = norm.rvs(loc=0, size=30, random_state=12345)
        >>> test    = norm.rvs(loc=0.5, size=30, random_state=12345)
        >>> my_df   = pd.DataFrame({"control": control,
                                    "test": test})
        >>> my_dabest_object = dabest.load(my_df, idx=("control", "test"))
        >>> my_dabest_object.cohens_d
        
        Notes
        -----
        Cohen's `d` is simply the mean of the control group subtracted from
        the mean of the test group.
        
        If `paired` is None, then the comparison(s) are unpaired; 
        otherwise the comparison(s) are paired.

        If the comparison(s) are unpaired, Cohen's `d` is computed with the following equation:
        
        .. math::
            
            d = \\frac{\\overline{x}_{Test} - \\overline{x}_{Control}} {\\text{pooled standard deviation}}
                
        
        For paired comparisons, Cohen's d is given by
        
        .. math::
            d = \\frac{\\overline{x}_{Test} - \\overline{x}_{Control}} {\\text{average standard deviation}}
            
        where :math:`\\overline{x}` is the mean of the respective group of observations, :math:`{Var}_{x}` denotes the variance of that group,
        
        .. math::
        
            \\text{pooled standard deviation} = \\sqrt{ \\frac{(n_{control} - 1) * {Var}_{control} + (n_{test} - 1) * {Var}_{test} } {n_{control} + n_{test} - 2} }
        
        and
        
        .. math::
        
            \\text{average standard deviation} = \\sqrt{ \\frac{{Var}_{control} + {Var}_{test}} {2}}
            
        The sample variance (and standard deviation) uses N-1 degrees of freedoms.
        This is an application of `Bessel's correction <https://en.wikipedia.org/wiki/Bessel%27s_correction>`_, and yields the unbiased
        sample variance.
        
        References:
            https://en.wikipedia.org/wiki/Effect_size#Cohen's_d
            https://en.wikipedia.org/wiki/Bessel%27s_correction
            https://en.wikipedia.org/wiki/Standard_deviation#Corrected_sample_standard_deviation
        """
        return self.__cohens_d
    
    
    @property
    def cohens_h(self):
        """
        Returns an :py:class:`EffectSizeDataFrame` for the standardized mean difference Cohen's `h`, its confidence interval, and relevant statistics, for all comparisons as indicated via the `idx` and `directional` argument in `dabest.load()`.

        Example
        -------
        >>> from scipy.stats import randint
        >>> import pandas as pd
        >>> import dabest
        >>> control = randint.rvs(0, 2, size=30, random_state=12345)
        >>> test    = randint.rvs(0, 2, size=30, random_state=12345)
        >>> my_df   = pd.DataFrame({"control": control,
                                    "test": test})
        >>> my_dabest_object = dabest.load(my_df, idx=("control", "test")
        >>> my_dabest_object.cohens_h

        Notes
        -----
        Cohen's 'h' uses the information of proportion in the control and test groups to calculate the distance between two proportions.
        It can be used to describe the difference between two proportions as "small", "medium", or "large".
        It can be used to determine if the difference between two proportions is "meaningful".

        A directional Cohen's 'h' is computed with the following equation:

        .. math::
            h = 2 * \\arcsin{\\sqrt{proportion_{Test}}} - 2 * \\arcsin{\\sqrt{proportion_{Control}}}

        For a non-directional Cohen's 'h', the equation is:
        .. math::
            h = \\abs{2 * \\arcsin{\\sqrt{proportion_{Test}}}} - \\abs{2 * \\arcsin{\\sqrt{proportion_{Control}}}}
        
        References:
            https://en.wikipedia.org/wiki/Cohen%27s_h
        """
        return self.__cohens_h


    @property  
    def hedges_g(self):
        """
        Returns an :py:class:`EffectSizeDataFrame` for the standardized mean difference Hedges' `g`, its confidence interval, and relevant statistics, for all comparisons as indicated via the `idx` and `paired` argument in `dabest.load()`.
        
        
        Example
        -------
        >>> from scipy.stats import norm
        >>> import pandas as pd
        >>> import dabest
        >>> control = norm.rvs(loc=0, size=30, random_state=12345)
        >>> test    = norm.rvs(loc=0.5, size=30, random_state=12345)
        >>> my_df   = pd.DataFrame({"control": control,
                                    "test": test})
        >>> my_dabest_object = dabest.load(my_df, idx=("control", "test"))
        >>> my_dabest_object.hedges_g
        
        Notes
        -----
        
        Hedges' `g` is :py:attr:`cohens_d` corrected for bias via multiplication with the following correction factor:
        
        .. math::
            \\frac{ \\Gamma( \\frac{a} {2} )} {\\sqrt{ \\frac{a} {2} } \\times \\Gamma( \\frac{a - 1} {2} )}
            
        where
        
        .. math::
            a = {n}_{control} + {n}_{test} - 2
            
        and :math:`\\Gamma(x)` is the `Gamma function <https://en.wikipedia.org/wiki/Gamma_function>`_.
            
        
        
        References:
            https://en.wikipedia.org/wiki/Effect_size#Hedges'_g
            https://journals.sagepub.com/doi/10.3102/10769986006002107
        """
        return self.__hedges_g
        
        
    @property    
    def cliffs_delta(self):
        """
        Returns an :py:class:`EffectSizeDataFrame` for Cliff's delta, its confidence interval, and relevant statistics, for all comparisons as indicated via the `idx` and `paired` argument in `dabest.load()`.
        
        
        Example
        -------
        >>> from scipy.stats import norm
        >>> import pandas as pd
        >>> import dabest
        >>> control = norm.rvs(loc=0, size=30, random_state=12345)
        >>> test    = norm.rvs(loc=0.5, size=30, random_state=12345)
        >>> my_df   = pd.DataFrame({"control": control,
                                    "test": test})
        >>> my_dabest_object = dabest.load(my_df, idx=("control", "test"))
        >>> my_dabest_object.cliffs_delta
        
        
        Notes
        -----
        
        Cliff's delta is a measure of ordinal dominance, ie. how often the values from the test sample are larger than values from the control sample.
        
        .. math::
            \\text{Cliff's delta} = \\frac{\\#({x}_{test} > {x}_{control}) - \\#({x}_{test} < {x}_{control})} {{n}_{Test} \\times {n}_{Control}}
            
            
        where :math:`\\#` denotes the number of times a value from the test sample exceeds (or is lesser than) values in the control sample. 
         
        Cliff's delta ranges from -1 to 1; it can also be thought of as a measure of the degree of overlap between the two samples. An attractive aspect of this effect size is that it does not make an assumptions about the underlying distributions that the samples were drawn from. 
        
        References:
            https://en.wikipedia.org/wiki/Effect_size#Effect_size_for_ordinal_data
            https://psycnet.apa.org/record/1994-08169-001
        """
        return self.__cliffs_delta


    @property
    def data(self):
        """
        Returns the pandas DataFrame that was passed to `dabest.load()`.
        """
        return self.__data


    @property
    def idx(self):
        """
        Returns the order of categories that was passed to `dabest.load()`.
        """
        return self.__idx
    

    @property
    def x1(self):
        return self.__x1


    @property
    def x1_level(self):
        return self.__x1_level


    @property
    def x2(self):
        return self.__x2


    @property
    def experiment(self):
        return self.__experiment
    

    @property
    def experiment_label(self):
        return self.__experiment_label


    @property
    def delta2(self):
        return self.__delta2


    @property
    def is_paired(self):
        """
        Returns the type of repeated-measures experiment.
        """
        return self.__is_paired


    @property
    def id_col(self):
        """
        Returns the id column declared to `dabest.load()`.
        """
        return self.__id_col


    @property
    def ci(self):
        """
        The width of the desired confidence interval.
        """
        return self.__ci


    @property
    def resamples(self):
        """
        The number of resamples used to generate the bootstrap.
        """
        return self.__resamples


    @property
    def random_seed(self):
        """
        The number used to initialise the numpy random seed generator, ie.
        `seed_value` from `numpy.random.seed(seed_value)` is returned.
        """
        return self.__random_seed


    @property
    def x(self):
        """
        Returns the x column that was passed to `dabest.load()`, if any.
        """
        return self.__x


    @property
    def y(self):
        """
        Returns the y column that was passed to `dabest.load()`, if any.
        """
        return self.__y


    @property
    def _xvar(self):
        """
        Returns the xvar in dabest.plot_data.
        """
        return self.__xvar


    @property
    def _yvar(self):
        """
        Returns the yvar in dabest.plot_data.
        """
        return self.__yvar


    @property
    def _plot_data(self):
        """
        Returns the pandas DataFrame used to produce the estimation stats/plots.
        """
        return self.__plot_data

    
    @property
    def proportional(self):
        """
        Returns the proportional parameter class.
        """
        return self.__proportional

    
    @property
    def mini_meta(self):
        """
        Returns the mini_meta boolean parameter.
        """
        return self.__mini_meta


    @property
    def _all_plot_groups(self):
        """
        Returns the all plot groups, as indicated via the `idx` keyword.
        """
        return self.__all_plot_groups

In [ ]:
#| export
class EffectSizeDataFrame(object):
    """A class that generates and stores the results of bootstrapped effect
    sizes for several comparisons."""

    def __init__(self, dabest, effect_size,
                 is_paired, ci=95, proportional=False,
                 resamples=5000, 
                 permutation_count=5000,
                 random_seed=12345, 
                 x1_level=None, x2=None, 
                 delta2=False, experiment_label=None,
                 mini_meta=False):
        """
        Parses the data from a Dabest object, enabling plotting and printing
        capability for the effect size of interest.
        """

        self.__dabest_obj        = dabest
        self.__effect_size       = effect_size
        self.__is_paired         = is_paired
        self.__ci                = ci
        self.__resamples         = resamples
        self.__permutation_count = permutation_count
        self.__random_seed       = random_seed
        self.__proportional      = proportional
        self.__x1_level          = x1_level
        self.__experiment_label  = experiment_label 
        self.__x2                = x2
        self.__delta2            = delta2 
        self.__mini_meta         = mini_meta


    def __pre_calc(self):
        import pandas as pd
        # from .misc_tools import print_greeting, get_varname

        idx  = self.__dabest_obj.idx
        dat  = self.__dabest_obj._plot_data
        xvar = self.__dabest_obj._xvar
        yvar = self.__dabest_obj._yvar

        out = []
        reprs = []

        for j, current_tuple in enumerate(idx):
            if self.__is_paired!="sequential":
                cname = current_tuple[0]
                control = dat[dat[xvar] == cname][yvar].copy()

            for ix, tname in enumerate(current_tuple[1:]):
                if self.__is_paired == "sequential":
                    cname = current_tuple[ix]
                    control = dat[dat[xvar] == cname][yvar].copy()
                test = dat[dat[xvar] == tname][yvar].copy()

                result = TwoGroupsEffectSize(control, test,
                                             self.__effect_size,
                                             self.__proportional,
                                             self.__is_paired,
                                             self.__ci,
                                             self.__resamples,
                                             self.__permutation_count,
                                             self.__random_seed)
                r_dict = result.to_dict()
                r_dict["control"]   = cname
                r_dict["test"]      = tname
                r_dict["control_N"] = int(len(control))
                r_dict["test_N"]    = int(len(test))
                out.append(r_dict)
                if j == len(idx)-1 and ix == len(current_tuple)-2:
                    if self.__delta2 and self.__effect_size == "mean_diff":
                        resamp_count = False
                        def_pval     = False
                    elif self.__mini_meta and self.__effect_size == "mean_diff":
                        resamp_count = False
                        def_pval     = False
                    else:
                        resamp_count = True
                        def_pval     = True
                else:
                    resamp_count = False
                    def_pval     = False

                text_repr = result.__repr__(show_resample_count=resamp_count,
                                            define_pval=def_pval)

                to_replace = "between {} and {} is".format(cname, tname)
                text_repr = text_repr.replace("is", to_replace, 1)

                reprs.append(text_repr)

        varname = get_varname(self.__dabest_obj)
        lastline = "To get the results of all valid statistical tests, " +\
        "use `{}.{}.statistical_tests`".format(varname, self.__effect_size)
        reprs.append(lastline)

        reprs.insert(0, print_greeting())

        self.__for_print = "\n\n".join(reprs)

        out_             = pd.DataFrame(out)

        columns_in_order = ['control', 'test', 'control_N', 'test_N',
                            'effect_size', 'is_paired',
                            'difference', 'ci',

                            'bca_low', 'bca_high', 'bca_interval_idx',
                            'pct_low', 'pct_high', 'pct_interval_idx',
                            
                            'bootstraps', 'resamples', 'random_seed',
                            
                            'permutations', 'pvalue_permutation', 'permutation_count', 'permutations_var',
                            
                            'pvalue_welch',
                            'statistic_welch',

                            'pvalue_students_t',
                            'statistic_students_t',

                            'pvalue_mann_whitney',
                            'statistic_mann_whitney',

                            'pvalue_brunner_munzel',
                            'statistic_brunner_munzel',

                            'pvalue_wilcoxon',
                            'statistic_wilcoxon',

                            'pvalue_paired_students_t',
                            'statistic_paired_students_t',

                            'pvalue_kruskal',
                            'statistic_kruskal',
                            'proportional_difference'
                           ]
        self.__results   = out_.reindex(columns=columns_in_order)
        self.__results.dropna(axis="columns", how="all", inplace=True)

        if self.__delta2 is True and self.__effect_size == "mean_diff":
            self.__delta_delta = DeltaDelta(self,
                                            self.__permutation_count,
                                            self.__ci)
            reprs.append(self.__delta_delta.__repr__(header=False))
        elif self.__delta2 is True and self.__effect_size != "mean_diff":
            self.__delta_delta = "Delta-delta is not supported for {}.".format(self.__effect_size)
        else:
            self.__delta_delta = "`delta2` is False; delta-delta is therefore not calculated."

        if self.__mini_meta is True and self.__effect_size == "mean_diff":
            self.__mini_meta_delta = MiniMetaDelta(self,
                                                     self.__permutation_count,
                                                     self.__ci)
            reprs.append(self.__mini_meta_delta.__repr__(header=False))
        elif self.__mini_meta is True and self.__effect_size != "mean_diff":
            self.__mini_meta_delta = "Weighted delta is not supported for {}.".format(self.__effect_size)
        else:
            self.__mini_meta_delta = "`mini_meta` is False; weighted delta is therefore not calculated."
        
        varname = get_varname(self.__dabest_obj)
        lastline = "To get the results of all valid statistical tests, " +\
        "use `{}.{}.statistical_tests`".format(varname, self.__effect_size)
        reprs.append(lastline)

        reprs.insert(0, print_greeting())

        self.__for_print = "\n\n".join(reprs)


    def __repr__(self):
        try:
            return self.__for_print
        except AttributeError:
            self.__pre_calc()
            return self.__for_print
            
            
            
    def __calc_lqrt(self):
        import lqrt
        import pandas as pd
        
        rnd_seed = self.__random_seed
        db_obj = self.__dabest_obj
        dat  = db_obj._plot_data
        xvar = db_obj._xvar
        yvar = db_obj._yvar
        delta2 = self.__delta2
        

        out = []

        for j, current_tuple in enumerate(db_obj.idx):
            if self.__is_paired != "sequential":
                cname = current_tuple[0]
                control = dat[dat[xvar] == cname][yvar].copy()

            for ix, tname in enumerate(current_tuple[1:]):
                if self.__is_paired == "sequential":
                    cname = current_tuple[ix]
                    control = dat[dat[xvar] == cname][yvar].copy()
                test = dat[dat[xvar] == tname][yvar].copy()
                
                if self.__is_paired:                    
                    # Refactored here in v0.3.0 for performance issues.
                    lqrt_result = lqrt.lqrtest_rel(control, test, 
                                            random_state=rnd_seed)
                    
                    out.append({"control": cname, "test": tname, 
                                "control_N": int(len(control)), 
                                "test_N": int(len(test)),
                                "pvalue_paired_lqrt": lqrt_result.pvalue,
                                "statistic_paired_lqrt": lqrt_result.statistic
                                })

                else:
                    # Likelihood Q-Ratio test:
                    lqrt_equal_var_result = lqrt.lqrtest_ind(control, test, 
                                                random_state=rnd_seed,
                                                equal_var=True)
                                                
                                                
                    lqrt_unequal_var_result = lqrt.lqrtest_ind(control, test, 
                                                random_state=rnd_seed,
                                                equal_var=False)
                                                
                    out.append({"control": cname, "test": tname, 
                                "control_N": int(len(control)), 
                                "test_N": int(len(test)),
                                
                                "pvalue_lqrt_equal_var"      : lqrt_equal_var_result.pvalue,
                                "statistic_lqrt_equal_var"   : lqrt_equal_var_result.statistic,
                                "pvalue_lqrt_unequal_var"    : lqrt_unequal_var_result.pvalue,
                                "statistic_lqrt_unequal_var" : lqrt_unequal_var_result.statistic,
                                })                     
        self.__lqrt_results = pd.DataFrame(out)


    def plot(self, color_col=None,

            raw_marker_size=6, es_marker_size=9,

            swarm_label=None, barchart_label=None, contrast_label=None, delta2_label=None,
            swarm_ylim=None, barchart_ylim=None, contrast_ylim=None, delta2_ylim=None,

            custom_palette=None, swarm_desat=0.5, halfviolin_desat=1,
            halfviolin_alpha=0.8, 

            face_color = None,
            #bar plot
            bar_label=None, bar_desat=0.9, bar_width = 0.5,bar_ylim = None,
            # error bar of proportion plot
            ci=None, err_color=None,

            float_contrast=True,
            show_pairs=True,
            show_delta2=True,
            show_mini_meta=True,
            group_summaries=None,
            group_summaries_offset=0.1,

            fig_size=None,
            dpi=100,
            ax=None,

            swarmplot_kwargs=None,
            barplot_kwargs=None,
            violinplot_kwargs=None,
            slopegraph_kwargs=None,
            reflines_kwargs=None,
            group_summary_kwargs=None,
            legend_kwargs=None):

        """
        Creates an estimation plot for the effect size of interest.
        

        Parameters
        ----------
        color_col : string, default None
            Column to be used for colors.
        raw_marker_size : float, default 6
            The diameter (in points) of the marker dots plotted in the
            swarmplot.
        es_marker_size : float, default 9
            The size (in points) of the effect size points on the difference
            axes.
        swarm_label, contrast_label, delta2_label : strings, default None
            Set labels for the y-axis of the swarmplot and the contrast plot,
            respectively. If `swarm_label` is not specified, it defaults to
            "value", unless a column name was passed to `y`. If
            `contrast_label` is not specified, it defaults to the effect size
            being plotted. If `delta2_label` is not specifed, it defaults to 
            "delta - delta"
        swarm_ylim, contrast_ylim, delta2_ylim : tuples, default None
            The desired y-limits of the raw data (swarmplot) axes, the
            difference axes and the delta-delta axes respectively, as a tuple. 
            These will be autoscaled to sensible values if they are not 
            specified. The delta2 axes and contrast axes should have the same 
            limits for y. When `show_delta2` is True, if both of the `contrast_ylim`
            and `delta2_ylim` are not None, then they must be specified with the 
            same values; when `show_delta2` is True and only one of them is specified,
            then the other will automatically be assigned with the same value.
            Specifying `delta2_ylim` does not have any effect when `show_delta2` is
            False. 
        custom_palette : dict, list, or matplotlib color palette, default None
            This keyword accepts a dictionary with {'group':'color'} pairings,
            a list of RGB colors, or a specified matplotlib palette. This
            palette will be used to color the swarmplot. If `color_col` is not
            specified, then each group will be colored in sequence according
            to the default palette currently used by matplotlib.
            Please take a look at the seaborn commands `color_palette`
            and `cubehelix_palette` to generate a custom palette. Both
            these functions generate a list of RGB colors.
            See:
            https://seaborn.pydata.org/generated/seaborn.color_palette.html
            https://seaborn.pydata.org/generated/seaborn.cubehelix_palette.html
            The named colors of matplotlib can be found here:
            https://matplotlib.org/examples/color/named_colors.html
        swarm_desat : float, default 1
            Decreases the saturation of the colors in the swarmplot by the
            desired proportion. Uses `seaborn.desaturate()` to acheive this.
        halfviolin_desat : float, default 0.5
            Decreases the saturation of the colors of the half-violin bootstrap
            curves by the desired proportion. Uses `seaborn.desaturate()` to
            acheive this.
        halfviolin_alpha : float, default 0.8
            The alpha (transparency) level of the half-violin bootstrap curves.            
        float_contrast : boolean, default True
            Whether or not to display the halfviolin bootstrapped difference
            distribution alongside the raw data.
        show_pairs : boolean, default True
            If the data is paired, whether or not to show the raw data as a
            swarmplot, or as slopegraph, with a line joining each pair of
            observations.
        show_delta2, show_mini_meta : boolean, default True
            If delta-delta or mini-meta delta is calculated, whether or not to 
            show the delta-delta plot or mini-meta plot.
        group_summaries : ['mean_sd', 'median_quartiles', 'None'], default None.
            Plots the summary statistics for each group. If 'mean_sd', then
            the mean and standard deviation of each group is plotted as a
            notched line beside each group. If 'median_quantiles', then the
            median and 25th and 75th percentiles of each group is plotted
            instead. If 'None', the summaries are not shown.
        group_summaries_offset : float, default 0.1
            If group summaries are displayed, they will be offset from the raw
            data swarmplot groups by this value. 
        fig_size : tuple, default None
            The desired dimensions of the figure as a (length, width) tuple.
        dpi : int, default 100
            The dots per inch of the resulting figure.
        ax : matplotlib.Axes, default None
            Provide an existing Axes for the plots to be created. If no Axes is
            specified, a new matplotlib Figure will be created.
        swarmplot_kwargs : dict, default None
            Pass any keyword arguments accepted by the seaborn `swarmplot`
            command here, as a dict. If None, the following keywords are
            passed to sns.swarmplot : {'size':`raw_marker_size`}.
        violinplot_kwargs : dict, default None
            Pass any keyword arguments accepted by the matplotlib `
            pyplot.violinplot` command here, as a dict. If None, the following
            keywords are passed to violinplot : {'widths':0.5, 'vert':True,
            'showextrema':False, 'showmedians':False}.
        slopegraph_kwargs : dict, default None
            This will change the appearance of the lines used to join each pair
            of observations when `show_pairs=True`. Pass any keyword arguments
            accepted by matplotlib `plot()` function here, as a dict.
            If None, the following keywords are
            passed to plot() : {'linewidth':1, 'alpha':0.5}.
        reflines_kwargs : dict, default None
            This will change the appearance of the zero reference lines. Pass
            any keyword arguments accepted by the matplotlib Axes `hlines`
            command here, as a dict. If None, the following keywords are
            passed to Axes.hlines : {'linestyle':'solid', 'linewidth':0.75,
            'zorder':2, 'color' : default y-tick color}.
        group_summary_kwargs : dict, default None
            Pass any keyword arguments accepted by the matplotlib.lines.Line2D
            command here, as a dict. This will change the appearance of the
            vertical summary lines for each group, if `group_summaries` is not
            'None'. If None, the following keywords are passed to
            matplotlib.lines.Line2D : {'lw':2, 'alpha':1, 'zorder':3}.
        legend_kwargs : dict, default None
            Pass any keyword arguments accepted by the matplotlib Axes
            `legend` command here, as a dict. If None, the following keywords
            are passed to matplotlib.Axes.legend : {'loc':'upper left',
            'frameon':False}.


        Returns
        -------
        A :class:`matplotlib.figure.Figure` with 2 Axes, if ``ax = None``.
        
        The first axes (accessible with ``FigName.axes[0]``) contains the rawdata swarmplot; the second axes (accessible with ``FigName.axes[1]``) has the bootstrap distributions and effect sizes (with confidence intervals) plotted on it.
        
        If ``ax`` is specified, the rawdata swarmplot is accessed at ``ax`` 
        itself, while the effect size axes is accessed at ``ax.contrast_axes``.
        See the last example below.
        

        Examples
        --------
        Create a Gardner-Altman estimation plot for the mean difference.

        >>> my_data = dabest.load(df, idx=("Control 1", "Test 1"))
        >>> fig1 = my_data.mean_diff.plot()

        Create a Gardner-Altman plot for the Hedges' g effect size.

        >>> fig2 = my_data.hedges_g.plot()

        Create a Cumming estimation plot for the mean difference.

        >>> fig3 = my_data.mean_diff.plot(float_contrast=True)

        Create a paired Gardner-Altman plot.

        >>> my_data_paired = dabest.load(df, idx=("Control 1", "Test 1"),
        ...                id_col = "ID", paired='baseline')
        >>> fig4 = my_data_paired.mean_diff.plot()

        Create a multi-group Cumming plot.

        >>> my_multi_groups = dabest.load(df, id_col = "ID", 
        ...                             idx=(("Control 1", "Test 1"),
        ...                                 ("Control 2", "Test 2")))
        >>> fig5 = my_multi_groups.mean_diff.plot()

        Create a shared control Cumming plot.

        >>> my_shared_control = dabest.load(df, id_col = "ID",
        ...                                 idx=("Control 1", "Test 1",
        ...                                          "Test 2", "Test 3"))
        >>> fig6 = my_shared_control.mean_diff.plot()
        
        Create a repeated meausures (against baseline) Slopeplot.

        >>> my_rm_baseline = dabest.load(df, id_col = "ID", paired = "baseline",
        ...                                 idx=("Control 1", "Test 1",
        ...                                          "Test 2", "Test 3"))
        >>> fig7 = my_rm_baseline.mean_diff.plot()

        Create a repeated meausures (sequential) Slopeplot.

        >>> my_rm_sequential = dabest.load(df, id_col = "ID", paired = "sequential",
        ...                                 idx=("Control 1", "Test 1",
        ...                                          "Test 2", "Test 3"))
        >>> fig8 = my_rm_sequential.mean_diff.plot()

        Creating estimation plots in individual panels of a figure.
        
        >>> f, axx = plt.subplots(nrows=2, ncols=2, figsize=(15, 15))
        >>> my_data.mean_diff.plot(ax=axx.flat[0])
        >>> my_data_paired.mean_diff.plot(ax=axx.flat[1])
        >>> my_shared_control.mean_diff.plot(ax=axx.flat[2])
        >>> my_shared_control.mean_diff.plot(ax=axx.flat[3], float_contrast=False)

        """

        from .plotter import EffectSizeDataFramePlotter

        if hasattr(self, "results") is False:
            self.__pre_calc()

        if self.__delta2:
            color_col = self.__x2

        # if self.__proportional:
        #     raw_marker_size = 0.01
            
        all_kwargs = locals()
        del all_kwargs["self"]

        out = EffectSizeDataFramePlotter(self, **all_kwargs)

        return out


    @property
    def proportional(self):
        """
        Returns the proportional parameter
        class.
        """
        return self.__proportional

    @property
    def results(self):
        """Prints all pairwise comparisons nicely."""
        try:
            return self.__results
        except AttributeError:
            self.__pre_calc()
            return self.__results



    @property
    def statistical_tests(self):
        results_df = self.results

        # Select only the statistics and p-values.
        stats_columns = [c for c in results_df.columns
                         if c.startswith("statistic") or c.startswith("pvalue")]

        default_cols = ['control', 'test', 'control_N', 'test_N',
                        'effect_size', 'is_paired',
                        'difference', 'ci', 'bca_low', 'bca_high']

        cols_of_interest = default_cols + stats_columns

        return results_df[cols_of_interest]


    @property
    def _for_print(self):
        return self.__for_print

    @property
    def _plot_data(self):
        return self.__dabest_obj._plot_data

    @property
    def idx(self):
        return self.__dabest_obj.idx

    @property
    def xvar(self):
        return self.__dabest_obj._xvar

    @property
    def yvar(self):
        return self.__dabest_obj._yvar

    @property
    def is_paired(self):
        return self.__is_paired

    @property
    def ci(self):
        """
        The width of the confidence interval being produced, in percent.
        """
        return self.__ci

    @property
    def x1_level(self):
        return self.__x1_level


    @property
    def x2(self):
        return self.__x2


    @property
    def experiment_label(self):
        return self.__experiment_label
    

    @property
    def delta2(self):
        return self.__delta2
    

    @property
    def resamples(self):
        """
        The number of resamples (with replacement) during bootstrap resampling."
        """
        return self.__resamples

    @property
    def random_seed(self):
        """
        The seed used by `numpy.seed()` for bootstrap resampling.
        """
        return self.__random_seed

    @property
    def effect_size(self):
        """The type of effect size being computed."""
        return self.__effect_size

    @property
    def dabest_obj(self):
        """
        Returns the `dabest` object that invoked the current EffectSizeDataFrame
        class.
        """
        return self.__dabest_obj

    @property
    def proportional(self):
        """
        Returns the proportional parameter
        class.
        """
        return self.__proportional
        
    @property
    def lqrt(self):
        """Returns all pairwise Lq-Likelihood Ratio Type test results 
        as a pandas DataFrame.
        
        For more information on LqRT tests, see https://arxiv.org/abs/1911.11922
        """
        try:
            return self.__lqrt_results
        except AttributeError:
            self.__calc_lqrt()
            return self.__lqrt_results
        
    
    @property
    def mini_meta(self):
        """
        Returns the mini_meta boolean parameter.
        """
        return self.__mini_meta

    
    @property
    def mini_meta_delta(self):
        """
        Returns the mini_meta results.
        """
        try:
            return self.__mini_meta_delta
        except AttributeError:
            self.__pre_calc()
            return self.__mini_meta_delta

    
    @property
    def delta_delta(self):
        """
        Returns the mini_meta results.
        """
        try:
            return self.__delta_delta
        except AttributeError:
            self.__pre_calc()
            return self.__delta_delta

In [ ]:
#| export
def print_greeting():
    # from .__init__ import __version__
    import datetime as dt
    import numpy as np

    line1 = "DABEST v{}".format(__version__)
    header = "".join(np.repeat("=", len(line1)))
    spacer = "".join(np.repeat(" ", len(line1)))

    now = dt.datetime.now()
    if 0 < now.hour < 12:
        greeting = "Good morning!"
    elif 12 < now.hour < 18:
        greeting = "Good afternoon!"
    else:
        greeting = "Good evening!"

    current_time = "The current time is {}.".format(now.ctime())

    return "\n".join([line1, header, spacer, greeting, current_time])


def get_varname(obj):
    matching_vars = [k for k,v in globals().items() if v is obj]
    if len(matching_vars) > 0:
        return matching_vars[0]
    else:
        return ""

In [ ]:
#| export
def load(data, idx=None, x=None, y=None, paired=None, id_col=None,
        ci=95, resamples=5000, random_seed=12345, proportional=False, 
        delta2 = False, experiment = None, experiment_label = None,
        x1_level = None, mini_meta=False):
    '''
    Loads data in preparation for estimation statistics.

    This is designed to work with pandas DataFrames.

    Parameters
    ----------
    data : pandas DataFrame
    idx : tuple
        List of column names (if 'x' is not supplied) or of category names
        (if 'x' is supplied). This can be expressed as a tuple of tuples,
        with each individual tuple producing its own contrast plot
    x : string or list, default None
        Column name(s) of the independent variable. This can be expressed as
        a list of 2 elements if and only if 'delta2' is True; otherwise it 
        can only be a string.
    y : string, default None
        Column names for data to be plotted on the x-axis and y-axis.
    paired : string, default None
        The type of the experiment under which the data are obtained
    id_col : default None.
        Required if `paired` is True.
    ci : integer, default 95
        The confidence interval width. The default of 95 produces 95%
        confidence intervals.
    resamples : integer, default 5000.
        The number of resamples taken to generate the bootstraps which are used
        to generate the confidence intervals.
    random_seed : int, default 12345
        This integer is used to seed the random number generator during
        bootstrap resampling, ensuring that the confidence intervals
        reported are replicable.
    proportional : boolean, default False. 
        TO INCLUDE MORE DESCRIPTION ABOUT DATA FORMAT
    delta2 : boolean, default False
        Indicator of delta-delta experiment
    experiment : String, default None
        The name of the column of the dataframe which contains the label of 
        experiments
    experiment_lab : list, default None
        A list of String to specify the order of subplots for delta-delta plots.
        This can be expressed as a list of 2 elements if and only if 'delta2' 
        is True; otherwise it can only be a string. 
    x1_level : list, default None
        A list of String to specify the order of subplots for delta-delta plots.
        This can be expressed as a list of 2 elements if and only if 'delta2' 
        is True; otherwise it can only be a string. 
    mini_meta : boolean, default False
        Indicator of weighted delta calculation.

    Returns
    -------
    A `Dabest` object.

    Example
    --------
    Load libraries.

    >>> import numpy as np
    >>> import pandas as pd
    >>> import dabest

    Create dummy data for demonstration.

    >>> np.random.seed(88888)
    >>> N = 10
    >>> c1 = sp.stats.norm.rvs(loc=100, scale=5, size=N)
    >>> t1 = sp.stats.norm.rvs(loc=115, scale=5, size=N)
    >>> df = pd.DataFrame({'Control 1' : c1, 'Test 1': t1})

    Load the data.

    >>> my_data = dabest_nbdev.load(df, idx=("Control 1", "Test 1"))

    '''

    return Dabest(data, idx, x, y, paired, id_col, ci, resamples, random_seed, proportional, delta2, experiment, experiment_label, x1_level, mini_meta)

## Examples

In [ ]:
#| hide
# import numpy as np
# import pandas as pd
# from scipy.stats import norm

Create dummy data for demonstration.

In [ ]:
#| hide
# np.random.seed(88888)
# N = 10
# c1 = norm.rvs(loc=100, scale=5, size=N)
# t1 = norm.rvs(loc=115, scale=5, size=N)
# df = pd.DataFrame({'Control 1' : c1, 'Test 1': t1})

Load the data.

In [ ]:
#| hide
# two_groups_unpaired = load(df, idx=("Control 1", "Test 1"), resamples=5000)
# two_groups_unpaired

DABEST v0.3.1
             
Good morning!
The current time is Sun Jan  8 01:19:17 2023.

Effect size(s) with 95% confidence intervals will be computed for:
1. Test 1 minus Control 1

5000 resamples will be used to generate the effect size bootstraps.